In [1]:
from hello.hello3 import *
import os
from datetime import datetime,timedelta

In [2]:
def hts(d):
    return int(d.timestamp()*1)

def fmt(d):
    return d.strftime("%m/%d/%y %I:%M%p")

def call(h, f, *args):
    while True:
        try:
            rv = f(*args)
        except NotLoggedInError:
            h.login()
        else:
            return rv

def download(h, d1, d2):
    s1 = hts(d1)
    s2 = hts(d2)
    fn = call(h, h.getReport, 'byDate', 'process_data', s1, s2)
    b  = call(h, h.getfile, fn)
    return b

In [9]:
h = HelloApp("192.168.1.12")
h.login()
b = h.getbatches()['test6']



d1 = b.start_time
end = b.stop_time
span = timedelta(hours=2)

path = "15L bio lab downloads\\"
os.makedirs(path, exist_ok=True)
print("Start:", fmt(d1))
print("End:  ", fmt(end))
while d1 < end:
    d2 = d1+span
    if d2 > end:
        d2 = end
    print("Downloading from %s to %s" % (fmt(d1),fmt(d2)))
    b = download(h, d1, d2)
    dfmt = "%m-%d-%y %H.%M"
    fn = path + "%s to %s.csv" % (d1.strftime(dfmt), d2.strftime(dfmt))
    with open(fn, 'wb') as f:
        f.write(b)
    d1 += span
print("Done")

Start: 07/11/18 03:53PM
End:   07/19/18 12:11PM
Done


In [22]:
path = "15L biolab downloads 180719\\"
files = os.listdir(path)
files.sort(key=lambda s: datetime.strptime(s.split(" to ")[0], "%m-%d-%y %H.%M"))

In [24]:
with open("compiled.csv", 'w') as bigf:
    for file in files:
        print("\rCompiling %s..."%file, end = "")
        with open(path + file, 'r') as f:
            chunk = f.read()
        bigf.write(chunk)
        if chunk[-1] != "\n":
            bigf.write("\n")

Compiling 07-19-18 11.53 to 07-19-18 12.11.csv...

In [114]:
from itertools import zip_longest

file = files[0]

def columnify(text):
    data = list(zip(*(l.split(",") for l in txt.splitlines())))
    return [list(l) for l in data]

def textify(data):
    data = list(zip_longest(*data, fillvalue=""))
    data = [",".join(l) for l in data]
    return "\n".join(data)

def merge(src, dst):
    if len(src) != len(dst):
        raise ValueError("Data length mismatch: %s != %s" % (len(src), len(dst)))
    for sr, ds in zip(src, dst):
        ds.extend(c for c in sr[1:] if c.strip())

with open(path+file, 'r') as f:
    header = f.readline()


data = [[h.strip()] for h in header.split(",")]

for file in files:
    with open(path+file, 'r') as f:
        txt = f.read()
    new = columnify(txt)
    print("\rMerging %s"%file, end="")
    merge(new, data)
txt = textify(data)
with open("compiled2.csv", 'w') as bigf:
    bigf.write(txt)
print("\ndone")

Merging 07-19-18 11.53 to 07-19-18 12.11.csv
done


In [111]:
data[135]

['TempPV(C)',
 '7/11/2018 3:54:25 PM',
 '7/11/2018 3:55:25 PM',
 '7/11/2018 3:56:26 PM',
 '7/11/2018 3:57:28 PM',
 '7/11/2018 3:58:29 PM',
 '7/11/2018 3:59:30 PM',
 '7/11/2018 4:00:30 PM',
 '7/11/2018 4:01:31 PM',
 '7/11/2018 4:02:32 PM',
 '7/11/2018 4:03:33 PM',
 '7/11/2018 4:04:34 PM',
 '7/11/2018 4:05:35 PM',
 '7/11/2018 4:06:36 PM',
 '7/11/2018 4:07:37 PM',
 '7/11/2018 4:08:38 PM',
 '7/11/2018 4:09:39 PM',
 '7/11/2018 4:10:40 PM',
 '7/11/2018 4:11:41 PM',
 '7/11/2018 4:12:42 PM',
 '7/11/2018 4:13:43 PM',
 '7/11/2018 4:14:44 PM',
 '7/11/2018 4:15:44 PM',
 '7/11/2018 4:16:45 PM',
 '7/11/2018 4:17:46 PM',
 '7/11/2018 4:18:47 PM',
 '7/11/2018 4:19:48 PM',
 '7/11/2018 4:20:49 PM',
 '7/11/2018 4:21:50 PM',
 '7/11/2018 4:22:51 PM',
 '7/11/2018 4:23:52 PM',
 '7/11/2018 4:24:53 PM',
 '7/11/2018 4:25:54 PM',
 '7/11/2018 4:26:55 PM',
 '7/11/2018 4:27:55 PM',
 '7/11/2018 4:28:56 PM',
 '7/11/2018 4:29:57 PM',
 '7/11/2018 4:30:58 PM',
 '7/11/2018 4:31:59 PM',
 '7/11/2018 4:33:00 PM',
 '7/11/2018

In [ ]:
from itertools import zip_longest, chain
class column():
    def __init__(x=(), y=()):
        self.x = x or []
        self.y = y or []

def grouper(iterable, n, fillvalue=None):
    "Collect data into fixed-length chunks or blocks"
    # grouper('ABCDEFG', 3, 'x') --> ABC DEF Gxx"
    args = [iter(iterable)] * n
    return zip_longest(*args, fillvalue=fillvalue)

def extract(f, ignore=None):
    ignore = ignore or ()
    data = f.read().splitlines()
    data = [l.split(",") for l in data]
    data = list(zip(*data))
    
    # I hate embedded metadata in a csv
    # this is the easiest way to parse it out
    
    for i, col in enumerate(data):
        if col[0] == "Start Time":
            found = True
            break
    else:
        found = False
    if found:
        data = data[:i-1]
    
    rv = {}
    for x, y, _ in grouper(data, 3, None):
        assert x[0] == y[0], (x[0], y[0])
        h = x[0]
        if h in ignore:
            continue
        x = [i for i in x if i != ""]
        y = [i for i in y if i != ""]
        rv[h] = x[1:], y[1:]
    return rv

def dmerge(src, dst):
    for k, (x, y) in src.items():
        if k not in dst:
            dst[k] = [],[]
        dst[k][0].extend(x)
        dst[k][1].extend(y)

def fcompile(files, ignore=None):
    data = {}
    for file in files:
        fp = path+file
        print("\rOpening %s                          " % file, end="")
        with open(fp, 'r') as f:
            d = extract(f, ignore)
        dmerge(d, data)
    print()
    return data
    
def write(fn, data):
    d = []
    for key in sorted(data):
        x, y = data[key]
        l1 = chain((key,), x)
        l2 = chain((key,), y)
        d.extend((l1, l2, []))
                  
#     for c in cols:
#         headers.extend((c, c, ""))
#         x,y = data[c]
#         d.extend((x, y, ()))
#     hs = ",".join(headers)
    
    d = list(zip_longest(*d, fillvalue=""))
    ds = "\n".join(",".join(l) for l in d)
    with open(fn, 'w') as f:
        f.write(ds)
        

In [ ]:
with open(path+files[0], 'r') as f:
    d = extract(f)
d['MFCCO2FlowFeedback(LPM)']

In [ ]:
ignore = ['MFCCO2MeasRaw(V)', 'MFCN2OutRaw(V).AO(V)', 'MFCCO2OutRaw.AO(V)', 'MFCN2MeasRaw(V)', 'MFCO2MeasRaw(V)', 'MFCAirOutRaw(V).AO(V)', 'MFCN2OutRaw(V).Period(iter)', 'MFCAirMeasRaw(V)']
data = fcompile(files, ignore)
write("15L Biolab Data 170630.csv", data)

In [ ]:
def mytimeit(f, n=1):
    from time import time
    start = time()
    for i in range(n):
        f()
    end = time()
    print("%d iteration(s) in %.2f seconds" % (n,end - start))

In [ ]:
def doit():
    data = fcompile(files, ignore)
    write("test2.csv", data)
mytimeit(doit)

In [ ]:
cols = list(data.keys())
headers = []
d = []
for c in cols:
    headers.extend((c, c, ""))
    x,y = data[c]
    d.extend((x, y, ()))
len(d)
len(headers)

In [ ]:
write("test.csv", data)

In [ ]:
from officelib.xllib import *
from officelib.const import xlconst as xlc

In [ ]:
xl = Excel()
wb = xl.RecentFiles(1).Open()

In [ ]:
ws = wb.Worksheets(1)
cells = ws.Cells
cell_range = cells.Range

In [ ]:
cell_range("CM1").EntireColumn._prop_map_get_.keys()

In [ ]:
cell_range("CM1").End(xlc.xlDown).End(xlc.xlDown).Address

In [ ]:
xlDown=xlc.xlDown
c = cell_range("CM1")
while True:
    c = c.End(xlDown)
    c2 = c.End(xlDown)
    if c2.Value != "pHPV":
        break
    c = c2

In [ ]:
c.Address

In [ ]:
def find_range(c):
    c1 = c
    c1v = c1.Value
    while True:
        c = c.End(xlDown)
        c2 = c.End(xlDown)
        if c2.Value != c1v:
            break
        c = c2
    return cell_range(c1, c.Offset(1,2))
def fix_data(d):
    c1v = d[0][0]
    return [(a,b) for a,b in d if a is not None and a != c1v]

In [ ]:
r = find_range(cell_range("CM1"))
d = r.Value
c1v = cell_range("CM1").Value
# only need to check a or b for None and header values
d2 = [(a, b) for a, b in d if a is not None and a != c1v]
r.Clear()
def paste(data, cell):
    rows = len(data)
    columns = len(data[0])
    br = cell.Offset(rows, columns)
    cell_range(cell, br).Value = data
paste(d2, cell_range("CM1"))

In [ ]:
paste(d2, cell_range("CM2"))

In [ ]:
len([r for r in d if r[0]])

In [ ]:
c = cell_range("A1")
while True:
    print("Fixing data for", c.Value)
    r = find_range(c)
    v = r.Value
    d = fix_data(v)
    hr = cell_range(c, c.Offset(1,2))
    header = hr.Value
    r.Clear()
    hr.Value = header
    paste(d, c.Offset(2,1))
    c = c.Offset(1, 4)
    if c.Value == "Start Time":
        break

In [ ]:
cells = wb.Worksheets("100%").Cells
cell_range = cells.Range
phxp_data = cell_range("CZ7")
diff_data = phxp_data.Offset(1,2)
phxp = cell_range("CY2")
diff = cell_range("DB4")
with screen_lock(xl):
    for i in range(30):
        i /= 100
        xp = 6+i
        phxp.Value = xp
        phxp_data.Value = xp
        diff_data.Value = diff.Value
        phxp_data = phxp_data.Offset(2,1)
        diff_data = diff_data.Offset(2,1)

# pH Model Analysis 3/14/17

In [ ]:
from officelib.xllib import *
xl = Excel()

In [ ]:
wb = xl.RecentFiles(2).Open()

In [ ]:
ws = wb.Worksheets("40%")
cells = ws.Cells
cell_range = cells.Range

In [ ]:
fudge = cell_range("CS2")
k = cell_range("DH5")
x = []
y = []
xl_nan = -2146826273
with screen_lock(xl):
    for i in range(30):
        fudge.Value = i/100
        v = k.Value
        if v != xl_nan:
            x.append(i/100)
            y.append(v)
        print("\rTesting fudge = %.2f" % (i/100), end="")

In [ ]:
%matplotlib
import matplotlib.pyplot as plt
plt.plot(x,y)

In [ ]:
c = xl.Selection
br = c.Offset(len(y), 2)
cell_range(c,br).Value = list(zip(x,y))

In [ ]:
hcpc = cell_range("DI3")
hcp = 3.4e-2
x = []
y = []
for offs in range(11):
    newv = hcp - offs/1000
    hcpc.Value = newv
    v = k.Value
    if v != xl_nan:
        x.append(newv)
        y.append(-v)
    print("\rTesting hcp = %.3f" % (newv), end="")

In [ ]:
plt.plot(x,y)

In [ ]:
import numpy as np
bicarb = cell_range("DG3")
bicarbs = np.linspace(0.018, 0.026, 10)
bicarb_value = bicarb.Value
x,y = [],[]
with screen_lock(xl):
    for bc in bicarbs:
        bicarb.Value = bc
        v = k.Value
        if v != xl_nan:
            x.append(bc)
            y.append(-v)
    bicarb.Value = bicarb_value